# Run before playing

## Card class definition

In [19]:
class Figure():
  def __init__(self, config, idx = 0):
    self.config = config
    self.idx = idx
    self.name = config['figure_str'][idx + 1] if (idx + 1) in config['figure_str'] else (idx + 1)
    self.str_max = len(str(config['figure_size']))

  def __str__(self):
    return f'{self.name:>{self.str_max}}'

In [20]:
class Color():
  def __init__(self, config, idx = 0):
    self.config = config
    self.idx = idx 
    self.name = config['color_str'][idx] if idx in config['color_str'] else idx + 1
    self.str_max = len(f"{config['color_size']}")

  def __str__(self):
    return f'{self.name:>{self.str_max}}'

In [21]:
class Card():
  def __init__(self, config, idx = 0, known = True):
    self.config = config
    self.idx = idx
    self.known = known
    self.figure = Figure(config, idx % config['figure_size']) 
    self.color = Color(config, idx // config['figure_size'])

  def hide(self):
    self.known = False

  def show(self):
    self.known = True

  def kwargs(self):
    return {'config': self.config, 'idx': self.idx, 'known': self.known}

  def __str__(self):
    if self.known:
      card_unicode = '1F0'
      if self.figure.idx < 13 and self.color.idx < 4:
        card_unicode += {0: 'A', 1: 'B', 2: 'D', 3: 'C'}[self.color.idx]
        card_unicode += f'{self.figure.idx + 1:x}'
      else:
        card_unicode += 'A0'
      print_ = f'{chr(int(card_unicode, 16))} {self.color}{self.figure}'
    else:
      print_ = '🂠 ? ?'
    return f"{print_:<{self.config['space_per_card']}}"
  
  def __eq__(self, other):
    if other == None:
      return False
    return (self.idx == other.idx) and (self.known == other.known)

## Move class definition

In [22]:
class Alf_Move():
  def __init__(self, group, stack = None, arg = None):
    self.group = group
    self.stack = stack if stack != None else group[0]
    self.arg = arg

  def reply(self, group, stack = None, arg = None):
    stack = stack if stack != None else group[0]
    if arg == None :
      arg = len(stack) - 1
    return Move(self, Alf_Move(group, stack, arg))

  def __str__(self):
    if self.group.name in ['Hand', 'Board'] and self.stack:
      return f'{self.stack[-1]}'
    return f'{self.stack}'

In [23]:
class Move():
  def __init__(self, ask, reply):
    self.ask = ask
    self.reply = reply
    self.get_str()

  def set_state(self, state):
    self.ask.group = state[self.ask.group.name]
    self.ask.stack = self.ask.group[self.ask.stack.idx]
    self.reply.group = state[self.reply.group.name]
    self.reply.stack = self.reply.group[self.reply.stack.idx]
    self.get_str()
  
  def do(self):
    cards = self.reply.group.get(idx = self.reply.stack.idx, arg = self.reply.arg)
    self.ask.group.put(idx = self.ask.stack.idx, cards = cards)
    if self.reply.group.name == 'Board':
      return self.reply.group.to_show != None

  def auto_score(self):
    if self.ask.group.name == 'Deck' or self.reply.group.name == 'Deck':
      return 0
    if self.ask.group.name == 'Sort':
      return 2
    if self.reply.group.name == 'Hand':
      return 3
    if self.reply.group.name == 'Sort':
      return 5
    unknown = len(self.reply.stack.unknown())
    if unknown:
      if self.reply.stack[self.reply.arg-1].known:
        return 6 
      return 2 - (unknown / 50)
    return 4

  def get_str(self):
    str_reply = f'{self.reply}'
    if self.reply.stack:
      str_reply =  f'{self.ask.arg}'
    self.str_ = f'{str_reply}-> {self.ask}'

  def  __str__(self):
    return self.str_

## Stacks class definition

In [24]:
class Stack(list):
  def __init__(self, config, cards = [], name = None, idx = 0):
    self.config = config
    super().__init__(cards)
    self.name = name
    self.idx = idx

  def kwargs(self):
    return {'config': self.config, 
            'cards': [Card(**card.kwargs()) for card in self], 
            'name': self.name, 
            'idx': self.idx}

  def hide(self):
    for card in self:
      card.hide()

  def show(self):
    for card in self:
      card.show()

  def unknown(self):
    return [Card(**card.kwargs()) for card in self if not card.known]
  
  def get(self, arg = None):
    if arg < 0:
      return [self.pop(- arg - 1)]
    cards = Stack(self.config, self[arg:])
    super().__init__(self[:arg])
    return cards

  def put(self, cards):
    self += cards

  def shuffle(self, seed):
    random.seed(seed)
    random.shuffle(self)

  def __hash__(self):
    hash_ = f'9{self.idx}'
    for card in self:
      hash_ += f'{card.idx:02d}'
    return hash(hash_)

  def __str__(self):
    if self:
      if self.name == 'Hand' and self.config['deck_in_hand']:
        print_ = ''
        for idx, card in enumerate(self):
          print_ += f"{f'{card}':<{self.config['space_per_card']}}"
          print_ += '\t' if (idx + 1) % self.config['card_per_line'] else '\n'
        return print_[:-1]
      return f"{f'{self[-1]}':<{self.config['space_per_card']}}"
    print_ = ''
    if self.name == 'Sort':
      print_ = f'{Color(self.config, self.idx)}'
    if not self.config['deck_in_hand']:
      if self.name == 'Deck':
        print_ = '▯ < <'
      if self.name == 'Hand':
        print_ = '▯'
    return f"{print_:<{self.config['space_per_card']}}"

In [25]:
class Group(list):
  def __init__(self, config, name = '', stacks = None, len = 1):
    self.config = config
    self.name = name
    stacks = stacks if stacks else [Stack(config, name=name, idx=idx) for idx in range(len)]
    super().__init__(stacks)
  
  def kwargs(self):
    return {'config': self.config, 
            'name': self.name, 
            'stacks': [Stack(**stack.kwargs()) for stack in self]}

  def get(self, idx = 0, arg = None):
    return self[idx].get(arg)
    
  def put(self, idx = 0, cards = None):
    self[idx].put(cards)

  def get_len(self):
    len_ = 0
    for stack in self:
      len_ += len(stack)
    return len_
    
  def __hash__(self):
    hash_ = ({'Deck': 80, 'Hand': 81, 'Sort': 82, 'Board': 83, None: 84}[self.name],)
    for stack in self:
      hash_ += (hash(stack),)
    return hash(hash_)

  def __str__(self):
    print = ''
    for stack in self:
      print += f'{stack}\t'
    return print[:-1]

In [26]:
class Deck(Group):
  def __init__(self, **group_kwargs):
    if 'name' not in group_kwargs:
      name = 'Deck'
      config = group_kwargs['config']
      size = config['figure_size'] * config['color_size']
      cards = [Card(config, idx) for idx in range(size)]
      stack = Stack(config, cards, name)
      stack.hide()
      stack.shuffle(config['seed'])
      group_kwargs.update({'name': name, 'stacks': [stack]})
    super().__init__(**group_kwargs)

  def put(self, idx = 0, cards= None):
    super().put(idx, cards)
    self[idx].reverse()
    self[idx].hide()

  def ask(self):
    return None if self[0] else [Alf_Move(self)]
  
  def reply(self, ask):
    if ask.group.name == 'Hand' and self[0]:
      return ask.reply(self)

In [27]:
class Hand(Group):
  def __init__(self, deck = None, **group_kwargs):
    if 'name' not in group_kwargs:
      name = 'Hand'
      config = group_kwargs['config']
      group_kwargs.update({'name': name})
      if config['deck_in_hand']:
        stack = Stack(config, deck.get(arg=0), name)
        stack.show()
        group_kwargs.update({'stacks': [stack]})
    super().__init__(**group_kwargs)
  
  def put(self, idx = 0, cards = None):
    super().put(idx, cards)
    self[0][-1].show()

  def ask(self):
    if not self.config['deck_in_hand']:
      return [Alf_Move(self)]

  def reply(self, ask):
    if self[0]:
      if self.config['deck_in_hand']:
        for idx, card in enumerate(self[0]):
          if card == ask.arg:
            return ask.reply(self, arg = - idx - 1)
      else:
        if self[0][-1] == ask.arg:
          return ask.reply(self)
        if ask.group.name == 'Deck':
          return ask.reply(self, arg = 0)

In [28]:
class Sort(Group):
  def __init__(self, **group_kwargs):
    if 'name' not in group_kwargs:
      name = 'Sort'
      config = group_kwargs['config']
      len = config['color_size']
      group_kwargs.update({'name': name, 'len': len})
    super().__init__(**group_kwargs)

  def ask(self):
    ask = []
    for idx, stack in enumerate(self):
      if len(stack) < self.config['figure_size']:
        next_idx = idx*self.config['figure_size'] + len(stack)
        ask.append(Alf_Move(self, stack, Card(self.config, next_idx)))
    return ask

  def reply(self, ask):
    for stack in self:
      if stack:
        if stack[-1] == ask.arg:
          return ask.reply(self, stack)

  def win(self):
    for stack in self:
      if len(stack) < self.config['figure_size']:
        return False
    return True

In [29]:
class Board(Group):
  def __init__(self, deck = None, to_show = None, **group_kwargs):
    if 'name' not in group_kwargs:
      name = 'Board'
      config = group_kwargs['config']
      size = config['board_stack_size']
      stacks = [Stack(config, deck.get(arg=len(deck[0])-idx-1), name, idx) for idx in range(size)]
      for stack in stacks:
        stack[-1].show()
      group_kwargs.update({'name': name, 'stacks': stacks})
    super().__init__(**group_kwargs)
    self.to_show = to_show

  def kwargs(self):
    return {'to_show': self.to_show, **super().kwargs()}

  def unknown(self):
    return [unknown for stack in self for unknown in stack.unknown()]

  def get(self, idx = 0, arg = None):
    cards = super().get(idx, arg)
    if self[idx]:
      if not self[idx][-1].known:
        self.to_show = idx
        if self.config['mode'] == 'Human':
          self.show()
    return cards

  def show(self, this_card = None):
    if self.to_show != None:
      new_card = self[self.to_show][-1]
      if this_card:
        for stack in self:
          for card in stack:
            if card == this_card:
              card, new_card = Card(**new_card.kwargs()), Card(**this_card.kwargs())
      new_card.show()
      self.to_show = None
    

  def ask(self):
    ask = []
    ask_for_king = False
    for stack in self:
      if stack:
        top_fig_idx = stack[-1].figure.idx
        top_col_idx = stack[-1].color.idx
        if top_fig_idx:
          for color in range(self.config['color_size']):
            if (top_col_idx + color) % 2:
              next_idx = color * self.config['figure_size'] + top_fig_idx - 1
              ask.append(Alf_Move(self, stack, Card(self.config, next_idx)))
      elif not ask_for_king:
        for color in range(self.config['color_size']):
          next_idx = (1+color) * self.config['figure_size'] - 1
          ask.append(Alf_Move(self, stack, Card(self.config, next_idx)))
        ask_for_king = True
    return ask

  def reply(self, ask):
    for stack in self:
      if stack:
        for idx, card in enumerate(stack):
          if card == ask.arg:
            if (card.figure.idx != self.config['figure_size'] - 1) or idx or (ask.group.name != 'Board'):
              if idx == len(stack) - 1:
                return ask.reply(self, stack)
              if ask.group.name == 'Board':
                return ask.reply(self, stack, idx)

  def __str__(self):
    max = 0
    for stack in self:
      if len(stack) > max:
        max = len(stack)
    if not max:
      return f"{'':<{self.config['space_per_card']}}"
    board = [['' for idx, stack in enumerate(self)] for _ in range(max)]
    for column, stack in enumerate(self):
      for line, card in enumerate(stack):
        board[line][column] = card
    
    print = ''
    for line in board:
      for card in line:
        print += f"{f'{card}':<{self.config['space_per_card']}}\t"
      print = print[:-1] + '\n'
    return print[:-1]

## Game class def

In [30]:
class State(dict):
  def __init__(self, config, groups = None):
    self.config = config
    if not groups:
      deck = Deck(config = config)
      sort = Sort(config = config)
      board = Board(deck = deck, config = config)
      hand = Hand(deck = deck, config = config)
      groups = {group.name: group for group in [deck, hand, sort, board]}
    super().__init__(groups)

  def kwargs(self):
    return {'config': self.config, 'groups': {group: self[group].__class__(**self[group].kwargs()) for group in self}}

  def get_moves(self):
    alf_move = []
    for group in self:
      ask = self[group].ask()
      if ask:
        alf_move += ask 

    moves = []
    for ask in alf_move:
      for group in self:
        reply = self[group].reply(ask)
        if reply:
          moves.append(reply)
    return moves
  
  def is_win(self):
    return self['Sort'].win()

  def unknown(self):
    return self['Board'].unknown()

  def get_score(self):
    score = 100
    score += -len(self.unknown())
    if self['Board'].to_show != None :
      score += len(self['Board'][self['Board'].to_show].unknown())
    score += self['Sort'].get_len()/10
    return score

  def __hash__(self):
    hash_ = ()
    for group in self:
      hash_ += (hash(self[group]),)
    return hash(hash_)


  def __str__(self):
    if self.config['deck_in_hand']:
      return f"{self['Sort']}\n\n{self['Board']}\n\n{self['Hand']}"
    return f"{self['Sort']}\t\t{self['Deck']}\t{self['Hand']}\n\n{self['Board']}"

In [31]:
from IPython.display import clear_output 
import time
import random 
from sys import maxsize

class Solitaire():
  def __init__(self, config):
    if not config['seed']:
      config['seed'] = random.randrange(maxsize)
    self.config = config
    self.data = []
    self.idx = 0
    while self.idx != self.config['game_limit']:
      clear_output()
      print(self.idx + 1, '/', self.config['game_limit'])
      self.dico_moves = {}
      self.next_moves = []
      self.past = []
      self.state = State(config)
      self.step = -1
      self.statue = 'PLAY'
      self.move = 'begining'
      self.time_ = time.time()
      self.step_time = self.time_
      while self.statue in ['PLAY', 'AUTO']:
        self.step += 1
        self.step_time = time.time()
        if self.config['mode'] == 'Human':
          self.Humain()
        if self.config['mode'] == 'Tree':
          self.Tree_main()
      self.data.append({'mode': self.config['mode'], 
                        'seed': self.config['seed'], 
                        'statue': self.statue, 
                        'step': self.step, 
                        'time': time.time() - self.time_})
      self.config['seed'] = self.new_seed()
      self.idx += 1
  
  def get_data(self):
    return self.data
    
  def Humain(self):
    if f'{self.state}' in self.dico_moves:
      moves = self.dico_moves[f'{self.state}']['moves']
    else:
      moves = self.state.get_moves()
      auto_score = [move.auto_score() + idx/1000 for idx, move in enumerate(moves)]
      moves = [idx for _, idx in sorted(zip(auto_score, moves))]
      self.dico_moves[f'{self.state}'] = {'moves': moves}

    if not moves:
      self.statue = 'LOSE'
    if self.state.is_win():
      self.statue = 'WIN'

    output = str(self)
    time.sleep(0.5)
    clear_output()
    print(output)

    if self.statue == 'PLAY':
      self.move = self.input_move(moves, not self.state.unknown())
      if self.statue == 'PLAY':
        self.move.do()
    if self.statue == 'AUTO':
      self.move = self.auto_move(moves)
      self.move.do()

  def input_move(self, moves, wining):
    print_ = '42: Auto Win\n' if wining else '\n'
    for idx, move in enumerate(moves):
      print_ += f"{idx}: {move}"
      print_ += '\t' if (idx + 1) % self.config['move_per_line'] else '\n'
    print(print_[:-1] + '\n69: Restart\n')
    while True:
      move_idx = input('Your move: ')
      if not move_idx:
        move_idx = '0'
      if move_idx.isdigit():
        move_idx = int(move_idx)
        if move_idx == 42 and wining:
          self.statue = 'AUTO'
          return 'auto'
        elif move_idx == 69:
          self.statue = 'LOST'
          return 'abord'
        elif move_idx in range(len(moves)):
          return moves[move_idx]
        else:
          print('Please input a possible move')
      else:
        print('Please input digits only')

  def auto_move(self, moves):
    if len(moves) == 1:
      return moves[0]
    return moves[1] if moves[1].ask.group.name == 'Sort' or moves[1].reply.group.name == 'Hand' else moves[0]

  def new_seed(self):
    if self.config['mode'] == 'Human':
      restart = input('[N] : New game or R : Restart the same ? ')
      if restart in ['R', 'Restart', 'r', 'restart', 're', 'Re', 'Restart the same', 'restart the same']:
        self.move = 'restarting'
        return self.config['seed']
      elif restart.isdigit():
        self.move = 'using seed'
        return int(restart)
    self.move = 'new game +'
    return random.randrange(maxsize)

  def Tree_main(self):
    if self.config['show'] and not self.step:
      clear_output()
      print(self)
    if (self.step > 300) or (self.state in self.past):
      scores_moves = None
    elif len(self.next_moves) > 1:
      scores_moves = [(self.next_moves, 0)]
    else:
      scores_moves = self.study()
      
    self.past.append(State(**self.state.kwargs()))
    if not scores_moves:
      self.statue = 'LOSE'
    if self.state.is_win():
      self.statue = 'WIN'

    if self.config['show']:
      output = f'{self}\n'
      idx = 0
      if scores_moves:
        for next_moves, score in scores_moves:
          idx += 1
          output += f"{next_moves[-1]} ({score:.2f})"
          output += '\t' if idx % self.config['move_per_line'] else '\n'
      clear_output()
      print(output[:-1])
      time.sleep(1)

    if self.statue == 'PLAY':
      self.next_moves, _ = scores_moves[0]
      self.move = self.next_moves.pop()
      self.move.set_state(self.state)
      if self.move.do():
        self.state['Board'].show()

  def study(self, state = None, past = None, level = 0):
    if not level :
      state = self.state
      past = [State(**past_state.kwargs()) for past_state in self.past]
      self.level_max = self.config['level_max'] if (len(state.unknown())>1) else 1
    if level >= self.level_max:
      return ([], state.get_score()) 
    if state in past:
      return ([], None) if level else None
    new_past = past + [State(**state.kwargs())]
    moves = state.get_moves()
    next = [self.make_move(state, move, idx) for idx, move in enumerate(moves)]
    next = sorted(next)
    scores_moves = []
    if next:
      for base_score, idx, next_state, move, new_card in next:
        if new_card:
          self.level_max = level + 2 if self.level_max > level + 2 else self.level_max
          score = base_score
          next_moves = [move]
        else:
          next_moves, score = self.study(next_state, new_past, level + 1)
          next_moves.append(move)
          if score != None:
            score = score*0.99
        if score != None:
          scores_moves.append((score, idx, next_moves))
    if not scores_moves:
      if self.state.is_win():
        return ([], 200) if level else None
      else: 
        return ([], None) if level else None
    sorted_moves = [(next_moves, score) for score, _, next_moves in sorted(scores_moves)]
    sorted_moves.reverse()
    if level:
      return sorted_moves[0]
    return sorted_moves

  def make_move(self, state, move, idx):
    next_state = State(**state.kwargs())
    move.set_state(next_state)
    new_card = move.do()
    return next_state.get_score(), idx, next_state, move, new_card

  def __str__(self):
    return f'''seed: {self.config['seed']:>18}  \
    move {self.step:<3}: {f'{self.move}':<8}  \
    time: {time.time() - self.step_time:.1f}  \
    ETA: {self.statue}\n
    {self.state}\n'''

# The game

In [48]:
config = {
'figure_size': 13,
'figure_str': {11: 'J', 12: 'Q', 13: 'K'},
'color_size': 4,
'color_str': {0: '♠', 1: '♡', 2: '♣', 3: '♢'},
'board_stack_size': 8,
'card_per_line': 10,
'space_per_card': 7,
'deck_in_hand': True,
'move_per_line': 3,
'seed': 5555,
'mode': 'Tree',
'game_limit': 7000,
'level_max': 2,
'show': False}

data = Solitaire(config).get_data()

clear_output()

step_time = 0
win_time = 0
lose_time = 0
win_step = 0
lose_step = 0
win = 0
for game in data:
  step_time += game['time'] / game['step']
  if game['statue'] == 'WIN':
    win += 1
    win_time += game['time']
    win_step += game['step']
  else:
    lose_time += game['time']
    lose_step += game['step']

step_time_avg = step_time / len(data)
print('step_time_avg :', step_time_avg)
win_rate = win / len(data)
print('win_rate :', 100*win_rate, '%')
if win_rate != 1:
  lose_time_avg = lose_time / (len(data)-win)
  print('lose_time_avg :', lose_time_avg)
  lose_step_avg = lose_step / (len(data)-win)
  print('lose_step_avg :', lose_step_avg)
if win_rate != 0:
  win_time_avg = win_time / win
  print('win_time_avg :', win_time_avg)
  win_step_avg = win_step / win
  print('win_step_avg :', win_step_avg)

step_time_avg : 0.15067324151702266
win_rate : 31.385714285714283 %
lose_time_avg : 19.510789495941104
lose_step_avg : 74.88340620445555
win_time_avg : 15.050010456732853
win_step_avg : 91.52162039144288


In [ ]:
config = {
'figure_size': 13,
'figure_str': {11: 'J', 12: 'Q', 13: 'K'},
'color_size': 4,
'color_str': {0: '♠', 1: '♡', 2: '♣', 3: '♢'},
'board_stack_size': 8,
'card_per_line': 10,
'space_per_card': 7,
'deck_in_hand': True,
'move_per_line': 3,
'seed': 5555,
'mode': 'Tree',
'game_limit': 7000,
'level_max': 2,
'show': False}

data = Solitaire(config).get_data()

clear_output()

step_time = 0
win_time = 0
lose_time = 0
win_step = 0
lose_step = 0
win = 0
for game in data:
  step_time += game['time'] / game['step']
  if game['statue'] == 'WIN':
    win += 1
    win_time += game['time']
    win_step += game['step']
  else:
    lose_time += game['time']
    lose_step += game['step']

step_time_avg = step_time / len(data)
print('step_time_avg :', step_time_avg)
win_rate = win / len(data)
print('win_rate :', 100*win_rate, '%')
if win_rate != 1:
  lose_time_avg = lose_time / (len(data)-win)
  print('lose_time_avg :', lose_time_avg)
  lose_step_avg = lose_step / (len(data)-win)
  print('lose_step_avg :', lose_step_avg)
if win_rate != 0:
  win_time_avg = win_time / win
  print('win_time_avg :', win_time_avg)
  win_step_avg = win_step / win
  print('win_step_avg :', win_step_avg)

6307 / 7000
